## Importing the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

In [3]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
# since there is no header, lets give column name
column_name = ["Review","Sentiment"]

In [5]:

data_yelp.columns = column_name

In [6]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
data_yelp.shape

(1000, 2)

In [8]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)

In [9]:
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [10]:
# Assigning column name
data_amazon.columns = column_name

In [11]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [12]:
data_amazon.shape

(1000, 2)

In [13]:
data_imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)

In [14]:
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [15]:
data_imdb.columns = column_name

In [16]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [17]:
data_imdb.shape

(748, 2)

In [18]:
# Append all the data in a single dataframe
data = data_yelp.append([data_amazon,data_imdb], ignore_index=True)

In [19]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [20]:
data.shape

(2748, 2)

In [21]:
# Checking distribution of sentiments
data["Sentiment"].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [22]:
# Checking for nulls
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [23]:
x = data['Review']       # Input dataset
y = data['Sentiment']    # Output dataset

## Data CLeaning

I'll revmove stopwords, punctuation as well as apply lemmatization

In [24]:
import string
punct = string.punctuation  # importing punctuations
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [25]:
# Importing Stop_words 

from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

Creating a function for data cleaning

In [26]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [27]:

def text_data_cleaning(sentence):
    doc = nlp(sentence)

    tokens = []         # list of tokens
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_token = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_token.append(token)
    return cleaned_token

# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemms form

In [28]:
# Testing on simple text
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

### Vectorization Feature Engineering (TF-IDF)

In [29]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [31]:
# tokenizer=text_data_cleaning, tokenization will be done according to this function
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

In [32]:
classifier = LinearSVC()

### Train the model

In [33]:
# Splitting the dataset into x and y 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [34]:
# Getting shape of the splitted test and train dataset
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2198,), (550,), (2198,), (550,))

### Fitting x_train and y_train

In [35]:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])
# it will first do verctorization and then it'll do classification

In [36]:
clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x00000266472E6D30>)),
                ('clf', LinearSVC())])

## Predicting the Test Set results

In [37]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [38]:
y_pred = clf.predict(x_test)

In [45]:
# Confusion Metrics
print("Confusion Metrics:\n",confusion_matrix(y_test, y_pred),'\n')


Confusion Metrics:
 [[202  77]
 [ 51 220]] 



In [46]:
# Classification Report
print("Classification Report:\n",classification_report(y_test, y_pred),'\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.81      0.77       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550
 



In [47]:
# Accuracy Score
print("Accuracy Score:",accuracy_score(y_test,y_pred)*100,'\n')

Accuracy Score: 76.72727272727272 



### Testing on new sentence

In [52]:
clf.predict(["It is a good day to go out"])

array([1], dtype=int64)

In [64]:
clf.predict(["Weater is not bad"])

array([0], dtype=int64)